In [2]:
library(Seurat)
library(tidyverse)
library(xlsx)

Attaching SeuratObject

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [ ]:
seuratobj=readRDS('MXMERZ002A/LargeData/seurat_obj.PostQC.rds')

In [3]:
# Prepare Metadata from excel
metadata_excel=read.xlsx('/mnt/MXMERZ002A/LargeData/2022-09-19_MXMERZ002A_Library_Info.xlsx', sheetName='metadata')

metadata=metadata_excel %>% mutate(Age=round(as.numeric(difftime(as.Date('2022/10/08'),date.of.birth, units = 'weeks')/52.25), 1)) %>%
mutate(name=as.integer(gsub("^Patient 0+", "", name))) %>% rename('patient'=name, 'days.since.apharesis'=days.since.collection) %>% 
select(patient, Age,sex, collection.day, days.since.apharesis, days.since.CAR.infusion, source, remission.after.CAR, SAMPLE_NAME, barcode) #note collection.day gets messed up when added to seuratmetadata since it's a Date object

In [6]:
seuratobj$orig.ident2=seuratobj$orig.ident
seuratobj$orig.ident=str_extract(seuratobj$orig.ident2, '^MXMERZ002A_\\d{2}')

In [8]:
# Left join metadata from excel
seuratobj@meta.data=seuratobj@meta.data %>% rownames_to_column('barcodes') %>% 
left_join(metadata, by=c('orig.ident'='SAMPLE_NAME')) %>% 
column_to_rownames('barcodes')

In [ ]:
seuratobj$wetlabid=seuratobj$orig.ident
seuratobj@meta.data=seuratobj@meta.data %>%unite(orig.ident, patient, days.since.apharesis, days.since.CAR.infusion, source, remove=F)
seuratobj$orig.ident=paste0('P',seuratobj$orig.ident)
seuratobj$orig.ident=factor(seuratobj$orig.ident, levels=str_sort(unique(seuratobj$orig.ident), numeric=T))

In [14]:
# WNN map
#PBMC
seuratobj_patient_list=SplitObject(seuratobj, split.by = "orig.ident")

In [15]:
wnnmap=function(pbmc3k){
pbmc3k <- SCTransform(pbmc3k, verbose = FALSE)
anchors <- FindTransferAnchors(
  reference = pbmc_reference,
  query = pbmc3k,
  normalization.method = "SCT",
  reference.reduction = "spca",
  dims = 1:50
)

pbmc3k <- TransferData(
  anchorset = anchors, 
  reference = pbmc_reference,
  query = pbmc3k,
  refdata = list(WNN_PBMC = "celltype.l2")
)}

In [ ]:
pbmc_reference <- LoadH5Seurat("/mnt/cnv/LargeData/pbmc_multimodal.h5seurat")

In [27]:
cellsources=lapply(seuratobj_patient_list, function(x) x$source[1])
pbsamples=seuratobj_patient_list[names(cellsources)[cellsources=='PB']]

In [ ]:
pbsamples=lapply(pbsamples, wnnmap)

In [30]:
saveRDS(pbsamples,'/mnt/MXMERZ002A/LargeData/pbmc_annotated_list.rds', compress=F )

In [31]:
bmsamples=seuratobj_patient_list[names(cellsources)[cellsources=='BM']]

In [ ]:
library(SeuratData)
# WNN map
# BM

In [33]:
bm <- LoadData(ds = "bmcite")

In [34]:
bm <- ScaleData(bm, assay = 'RNA')
bm <- RunSPCA(bm, assay = 'RNA', graph = 'wsnn')

Centering and scaling data matrix

Computing sPCA transformation



In [35]:
bm <- FindNeighbors(
  object = bm,
  reduction = "spca",
  dims = 1:50,
  graph.name = "spca.annoy.neighbors", 
  k.param = 50,
  cache.index = TRUE,
  return.neighbor = TRUE,
  l2.norm = TRUE
)

Computing nearest neighbors

Only one graph name supplied, storing nearest-neighbor graph only



In [36]:
changeassaydata=function(x) {DefaultAssay(x)='RNA' 
                            return(x)}
bmsamples=lapply(bmsamples, changeassaydata)

In [ ]:
anchors <- list()
for (i in 1:length(bmsamples)) {
  anchors[[i]] <- FindTransferAnchors(
    reference = bm,
    query = bmsamples[[i]],
    k.filter = NA,
    reference.reduction = "spca", 
    reference.neighbors = "spca.annoy.neighbors", 
    dims = 1:50
  )
}

In [ ]:
for (i in 1:length(bmsamples)) {
    bmsamples[[i]] <- TransferData( anchorset =  anchors[[i]],
                                   reference = bm,
                                   query = bmsamples[[i]],
                                   refdata = list(WNN_BM = "celltype.l2"))
}

In [ ]:
saveRDS(bmsamples,'/mnt/MXMERZ002A/LargeData/bm_annotated_list.rds', compress=F )

In [2]:
pbsamples=readRDS('/mnt/MXMERZ002A/LargeData/pbmc_annotated_list.rds')
bmsamples=readRDS('/mnt/MXMERZ002A/LargeData/bm_annotated_list.rds')

In [3]:
annotated_list=c(pbsamples, bmsamples)

In [4]:
seuratobj=merge(annotated_list[[1]], annotated_list[2:length(annotated_list)])

Warning message:
“Attempting to merge an SCTAssay with another Assay type 
Converting all to standard Assay objects.”


In [5]:
seuratobj@meta.data=seuratobj@meta.data %>% mutate(WNN_l2=ifelse(source=='PB', predicted.WNN_PBMC, predicted.WNN_BM))

In [5]:
seuratobj@meta.data=seuratobj@meta.data %>% mutate(WNN_l2=ifelse(WNN_l2=='Plasmablast','Plasma cells', WNN_l2)) #Change plasmablast to plasma cell for bmcite

In [8]:
# Add cell annotation
get_short_name <- function(t){
    #t <- gsub('Prog_RBC','Prog_RBC',t)
    t <- ifelse(grepl('B ', t), 'B cells', t)
    t <- ifelse(grepl('CD4 ', t), 'CD4 T cells', t)
    t <- ifelse(grepl('CD8 ', t), 'CD8 T cells', t)
    t <- ifelse(grepl('cDC', t), 'cDC', t)
    t <- ifelse(grepl('NK', t), 'NK', t)
    t = ifelse(grepl('MAIT|ILC|HSPC|ASDC|Doublet|Platelet|Eryth', t), 'Others', t)
    t = ifelse(grepl('CD14 |CD16 ', t), 'Monocytes', t)
    t = ifelse(grepl('CD14 |CD16 ', t), 'Monocytes', t)
    t <- ifelse(grepl('Plasmablast', t), 'Plasma cells', t)
    return(t)
}

get_short_name3 <- function(t){ #For BMCite reference
    #t <- gsub('Prog_RBC','Prog_RBC',t)
    t <- gsub('Prog_Mk|HSC|LMPP','HSPC',t)
    #t <- gsub('GMP','GMP',t)
    t <- gsub('CD14 Mono','Monocytes',t)
    t <- gsub('CD16 Mono','Monocytes',t)
    t <- gsub('cDC2|cDC1','cDC',t)
    t <- gsub('Prog_B 1|Prog_B 2','Prog_B',t) # pre-B
    t <- gsub('Memory B|Naive B','B cells',t) 
    t <- gsub('Plasma Cells|Plasmablast','Plasma cells',t)
    t <- gsub('CD4 Naive|CD4 Memory','CD4 T cells',t)
    t <- gsub('CD8 Effector_1|CD8 Effector_2|CD8 Memory_1|CD8 Memory_2|CD8 Naive','CD8 T cells',t)
    t <- gsub('Treg','Treg',t)
    t <- gsub('NK|CD56 bright NK','NK',t)
    return(t)
}

harmonise_name2 <- function(t){
    #t <- gsub('Prog_RBC','Prog_RBC',t)
    t <- ifelse(grepl('cDC', t), 'DC', t)
    t <- ifelse(grepl('Dendritic cells', t), 'DC', t)
    t <- ifelse(grepl('Natural killer cells', t), 'NK', t)
    t <- ifelse(grepl('Plasmacytoid dendritic cells', t), 'pDC', t)
    t <- ifelse(grepl('Plasmablast', t), 'Plasma cells', t)
    return(t)
}

In [20]:
WNN_agg_l2=ifelse(seuratobj$source=='PB',seuratobj$predicted.WNN_PBMC, seuratobj$predicted.WNN_BM)
seuratobj$WNN_agg_l2=WNN_agg_l2

In [19]:
pb_annot=sapply(seuratobj$predicted.WNN_PBMC, get_short_name)
bm_annot=sapply(seuratobj$predicted.WNN_BM, get_short_name3)

In [21]:
WNN_agg=ifelse(seuratobj$source=='PB',pb_annot, bm_annot)
seuratobj$WNN_agg=WNN_agg

In [25]:
DefaultAssay(seuratobj)='RNA'
seuratobj=NormalizeData(seuratobj)

In [6]:
saveRDS(seuratobj, '/mnt/MXMERZ002A/LargeData/seuratobj_1.rds', compress = F)